# • Reconciliation Quick Start

<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/nbs/examples/TourismSmall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Large collections of time series organized into structures at different aggregation levels often require their forecasts to follow their aggregation constraints, which poses the challenge of creating novel algorithms capable of coherent forecasts.

The `HierarchicalForecast` package provides a wide collection of Python implementations of hierarchical forecasting algorithms that follow classic hierarchical reconciliation.

In [ ]:
%%capture
!pip install hierarchicalforecast
!pip install -U numba statsforecast datasetsforecast

In [ ]:
import numpy as np
import pandas as pd

#obtain hierarchical data
from datasetsforecast.hierarchical import HierarchicalData

# compute base forecast no coherent
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive

#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut

/usr/local/lib/python3.8/dist-packages/statsforecast/core.py:21: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In this example we will use the `TourismSmall` dataset. The following cell gets the time series for the different levels in the hierarchy, the summing matrix `S` which recovers the full dataset from the bottom level hierarchy and the indices of each hierarchy denoted by `tags`.

In [ ]:
Y_df, S_df, tags = HierarchicalData.load('./data', 'TourismSmall')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

100%|██████████| 1.30M/1.30M [00:00<00:00, 3.45MiB/s]


In [ ]:
Y_df.head()

,unique_id,ds,y
0,total,1998-03-31,84503
1,total,1998-06-30,65312
2,total,1998-09-30,72753
3,total,1998-12-31,70880
4,total,1999-03-31,86893


In [ ]:
S_df.iloc[:5, :5]

,nsw-hol-city,nsw-hol-noncity,vic-hol-city,vic-hol-noncity,qld-hol-city
total,1.0,1.0,1.0,1.0,1.0
hol,1.0,1.0,1.0,1.0,1.0
vfr,0.0,0.0,0.0,0.0,0.0
bus,0.0,0.0,0.0,0.0,0.0
oth,0.0,0.0,0.0,0.0,0.0


In [ ]:
tags

{'Country': array(['total'], dtype=object),
 'Country/Purpose': array(['hol', 'vfr', 'bus', 'oth'], dtype=object),
 'Country/Purpose/State': array(['nsw-hol', 'vic-hol', 'qld-hol', 'sa-hol', 'wa-hol', 'tas-hol',
        'nt-hol', 'nsw-vfr', 'vic-vfr', 'qld-vfr', 'sa-vfr', 'wa-vfr',
        'tas-vfr', 'nt-vfr', 'nsw-bus', 'vic-bus', 'qld-bus', 'sa-bus',
        'wa-bus', 'tas-bus', 'nt-bus', 'nsw-oth', 'vic-oth', 'qld-oth',
        'sa-oth', 'wa-oth', 'tas-oth', 'nt-oth'], dtype=object),
 'Country/Purpose/State/CityNonCity': array(['nsw-hol-city', 'nsw-hol-noncity', 'vic-hol-city',
        'vic-hol-noncity', 'qld-hol-city', 'qld-hol-noncity',
        'sa-hol-city', 'sa-hol-noncity', 'wa-hol-city', 'wa-hol-noncity',
        'tas-hol-city', 'tas-hol-noncity', 'nt-hol-city', 'nt-hol-noncity',
        'nsw-vfr-city', 'nsw-vfr-noncity', 'vic-vfr-city',
        'vic-vfr-noncity', 'qld-vfr-city', 'qld-vfr-noncity',
        'sa-vfr-city', 'sa-vfr-noncity', 'wa-vfr-city', 'wa-vfr-noncity',
     

We split the dataframe in train/test splits.

In [ ]:
Y_test_df = Y_df.groupby('unique_id').tail(12)
Y_train_df = Y_df.drop(Y_test_df.index)

In [ ]:
Y_test_df = Y_test_df.set_index('unique_id')
Y_train_df = Y_train_df.set_index('unique_id')

The following cell computes the *base forecast* for each time series using the `auto_arima` and `naive` models. Observe that `Y_hat_df` contains the forecasts but they are not coherent.

In [ ]:
%%capture
fcst = StatsForecast(
    df=Y_train_df, 
    models=[AutoARIMA(season_length=12), Naive()], 
    freq='M', 
    n_jobs=-1
)
Y_hat_df = fcst.forecast(h=12)

In [ ]:
# S
np.save(open('S_tensor.npy', 'wb'), S_df)
S_df

,nsw-hol-city,nsw-hol-noncity,vic-hol-city,vic-hol-noncity,qld-hol-city,qld-hol-noncity,sa-hol-city,sa-hol-noncity,wa-hol-city,wa-hol-noncity,...,qld-oth-city,qld-oth-noncity,sa-oth-city,sa-oth-noncity,wa-oth-city,wa-oth-noncity,tas-oth-city,tas-oth-noncity,nt-oth-city,nt-oth-noncity
total,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
hol,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
vfr,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
oth,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wa-oth-noncity,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
tas-oth-city,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
tas-oth-noncity,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
nt-oth-city,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
# y_hat
y_i = Y_hat_df.reset_index()
y_hat = pd.pivot(y_i, index='unique_id', columns='ds', values='AutoARIMA').to_numpy()
np.save(open('yhat_tensor.npy', 'wb'), y_hat)


In [ ]:
# gt
gt_i = Y_test_df.reset_index()
gt = pd.pivot(gt_i, index='unique_id', columns='ds', values='y').to_numpy()
np.save(open('gt_tensor.npy', 'wb'), gt)

In [ ]:
# parent list
node_map = {}
node_map_inv = {}

i = 0

for l in ['Country/Purpose/State/CityNonCity', 'Country/Purpose/State', 'Country/Purpose', 'Country']:
    for c in tags[l]:
        node_map[i] = c
        node_map_inv[c] = i
        i += 1
        

In [ ]:
def findParent(i):
  if i in tags['Country/Purpose/State/CityNonCity']:
    cut = i.rfind('-')
    return i[:cut]
  if i in tags['Country/Purpose/State']:
    cut = i.rfind('-')
    return i[cut+1:]
  if i in tags['Country/Purpose']:
    return 'total'
print(findParent('nsw-hol'))

hol


In [ ]:
parent_list = []

for i, ro in S_df.iterrows():
    if i in tags['Country/Purpose/State/CityNonCity']:
      lv3 = i
      lv2 = findParent(i)
      lv1 = findParent(lv2)
      lv0 = findParent(lv1)
      parent_list.append([node_map_inv[lv3], node_map_inv[lv2], node_map_inv[lv1], node_map_inv[lv0]])
    elif i in tags['Country/Purpose/State']:
      lv2 = i
      lv1 = findParent(lv2)
      lv0 = findParent(lv1)
      parent_list.append([node_map_inv[lv2], node_map_inv[lv1], node_map_inv[lv0], -1])
    elif i in tags['Country/Purpose']:
      lv1 = i
      lv0 = findParent(lv1)
      parent_list.append([node_map_inv[lv1], node_map_inv[lv0], -1, -1])
    else:
      parent_list.append([node_map_inv[i], -1, -1, -1])
parent_list = sorted(parent_list, key=lambda x: x[0])
    

In [ ]:
parent_list

[[0, 56, 84, 88],
 [1, 56, 84, 88],
 [2, 57, 84, 88],
 [3, 57, 84, 88],
 [4, 58, 84, 88],
 [5, 58, 84, 88],
 [6, 59, 84, 88],
 [7, 59, 84, 88],
 [8, 60, 84, 88],
 [9, 60, 84, 88],
 [10, 61, 84, 88],
 [11, 61, 84, 88],
 [12, 62, 84, 88],
 [13, 62, 84, 88],
 [14, 63, 85, 88],
 [15, 63, 85, 88],
 [16, 64, 85, 88],
 [17, 64, 85, 88],
 [18, 65, 85, 88],
 [19, 65, 85, 88],
 [20, 66, 85, 88],
 [21, 66, 85, 88],
 [22, 67, 85, 88],
 [23, 67, 85, 88],
 [24, 68, 85, 88],
 [25, 68, 85, 88],
 [26, 69, 85, 88],
 [27, 69, 85, 88],
 [28, 70, 86, 88],
 [29, 70, 86, 88],
 [30, 71, 86, 88],
 [31, 71, 86, 88],
 [32, 72, 86, 88],
 [33, 72, 86, 88],
 [34, 73, 86, 88],
 [35, 73, 86, 88],
 [36, 74, 86, 88],
 [37, 74, 86, 88],
 [38, 75, 86, 88],
 [39, 75, 86, 88],
 [40, 76, 86, 88],
 [41, 76, 86, 88],
 [42, 77, 87, 88],
 [43, 77, 87, 88],
 [44, 78, 87, 88],
 [45, 78, 87, 88],
 [46, 79, 87, 88],
 [47, 79, 87, 88],
 [48, 80, 87, 88],
 [49, 80, 87, 88],
 [50, 81, 87, 88],
 [51, 81, 87, 88],
 [52, 82, 87, 88],
 [5

In [ ]:
np.save(open('parent.npy', 'wb'), np.array(parent_list))

In [ ]:
# p
top_down_tensor = np.zeros_like(gt)
for l in parent_list:
    i = l[0]
    for c in [l[3], l[2], l[1]]:
        if c != -1: break
    top_down_tensor[i, :] = gt[i, :] / (gt[c, :] + 1e-9)
top_down_tensor

array([[ 9, 12, 13, ..., 10, 15, 11],
       [44, 38, 38, ..., 32, 43, 33],
       [ 3,  3,  3, ...,  3,  4,  3],
       ...,
       [ 2,  2,  2, ...,  2,  2,  2],
       [ 1,  1,  1, ...,  1,  1,  1],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [ ]:
np.save(open('top_down_tensor.npy', 'wb'), top_down_tensor)

In [ ]:
level_2_tensor = np.zeros_like(gt)

for l in parent_list:
    nz = list(filter(lambda x: x!=-1, l))
    if len(nz) == 1: continue
    i = l[0]
    c = l[-2]
    level_2_tensor[i, :] = gt[i, :] / (gt[c, :] + 1e-9)
level_2_tensor

array([[ 26,  31,  49, ...,  52,  29,  32],
       [119,  93, 139, ..., 159,  79,  96],
       [  9,   9,  13, ...,  16,   8,   9],
       ...,
       [  2,   2,   2, ...,   2,   2,   2],
       [  1,   1,   1, ...,   1,   1,   1],
       [  0,   0,   0, ...,   0,   0,   0]])

In [ ]:
np.save(open('level_2_tensor.npy', 'wb'), level_2_tensor)

In [ ]:
level_3_tensor = np.zeros_like(gt)

for l in parent_list:
    nz = list(filter(lambda x: x!=-1, l))
    if len(nz) <= 2: continue
    i = l[0]
    c = l[-3]
    level_3_tensor[i, :] = gt[i, :] / (gt[c, :] + 1e-9)

In [ ]:
np.save(open('level_3_tensor.npy', 'wb'), level_3_tensor)

In [ ]:
N_CHUNCKS = 8

for t, n in [(y_hat, 'yhat_tensor'),
        (gt, 'gt_tensor'),
        (top_down_tensor, 'top_down_tensor'),
        (level_2_tensor, 'level_2_tensor'),
        (level_3_tensor, 'level_3_tensor')]:
  l = np.array_split(t, N_CHUNCKS)
  for i, p in enumerate(l):
    np.save(open('mpi/' + n + '_' + str(i) + '.npy', 'wb'), p)

The following cell makes the previous forecasts coherent using the `HierarchicalReconciliation` class. The used methods to make the forecasts coherent are:
- `BottomUp`: The reconciliation of the method is a simple addition to the upper levels.
- `TopDown`: The second method constrains the base-level predictions to the top-most aggregate-level serie and then distributes it to the disaggregate series through the use of proportions. 
- `MiddleOut`: Anchors the base predictions in a middle level.

In [ ]:
reconcilers = [
    BottomUp(),
    TopDown(method='forecast_proportions'),
    MiddleOut(middle_level='Country/Purpose/State', 
              top_down_method='forecast_proportions')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_train_df, 
                          S=S_df, tags=tags)

The `HierarchicalForecast` package includes the `HierarchicalEvaluation` class to evaluate the different hierarchies and also is capable of compute scaled metrics compared to a benchmark model.

In [ ]:
def mse(y, y_hat):
    return np.mean((y-y_hat)**2)

evaluator = HierarchicalEvaluation(evaluators=[mse])
evaluation = evaluator.evaluate(
        Y_hat_df=Y_rec_df, Y_test_df=Y_test_df, 
        tags=tags, benchmark='Naive'
)
evaluation.filter(like='ARIMA', axis=1).T

### References
- [Orcutt, G.H., Watts, H.W., & Edwards, J.B.(1968). Data aggregation and information loss. The American 
Economic Review, 58 , 773{787)](http://www.jstor.org/stable/1815532).
- [Disaggregation methods to expedite product line forecasting. Journal of Forecasting, 9 , 233–254. 
doi:10.1002/for.3980090304](https://onlinelibrary.wiley.com/doi/abs/10.1002/for.3980090304).<br>
- [An investigation of aggregate variable time series forecast strategies with specific subaggregate 
time series statistical correlation. Computers and Operations Research, 26 , 1133–1149. 
doi:10.1016/S0305-0548(99)00017-9](https://doi.org/10.1016/S0305-0548(99)00017-9).
- [Hyndman, R.J., & Athanasopoulos, G. (2021). "Forecasting: principles and practice, 3rd edition: 
Chapter 11: Forecasting hierarchical and grouped series.". OTexts: Melbourne, Australia. OTexts.com/fpp3 
Accessed on July 2022.](https://otexts.com/fpp3/hierarchical.html)